In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11773675436420871614
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2253809255
locality {
  bus_id: 1
  links {
  }
}
incarnation: 469850970935110977
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [4]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import Model

In [5]:
base_model = VGG16(weights='imagenet', include_top=True)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

In [7]:
path = 'C:/Users/sanghui/Desktop/toyproject/DL_Project_5/hsh/data/2_15_crop'

In [8]:
with tf.device('/device:CPU:0'):
    # load data
    img_datas = []

    for filename in tqdm_notebook(os.listdir(path)):
        file_path = path + '/' + filename
        src = cv2.imread(file_path, cv2.IMREAD_COLOR)
        img_datas.append(src)

    img_datas = np.array(img_datas)
    img_datas.shape
    

C:\Users\sanghui\AppData\Local\Temp/ipykernel_10200/2630330009.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for filename in tqdm_notebook(os.listdir(path)):


  0%|          | 0/51260 [00:00<?, ?it/s]

In [ ]:
# samples = np.random.choice(range(len(img_datas)), 10000, replace=False)

In [ ]:
# samples

In [9]:
with tf.device('/device:CPU:0'):
    features = []

    for img in tqdm_notebook(img_datas):
        img_data = np.expand_dims(img / 255.0, axis=0)
        img_data = preprocess_input(img_data)

        features.append(model.predict(img_data)[0])

C:\Users\sanghui\AppData\Local\Temp/ipykernel_10200/4082712732.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for img in tqdm_notebook(img_datas):


  0%|          | 0/51260 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
feature_vectors = np.array(features)

In [ ]:
# elbow score
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

km = KMeans()
visualizer = KElbowVisualizer(km, k=(1,20))
visualizer.fit(feature_vectors)
visualizer.show()